<a href="https://colab.research.google.com/github/VojtasekP/NAEX_reaction_time/blob/main/reaction_time_factorial_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reaction Time Experiment: 2^k Factorial Design with Blocking

This notebook analyzes a reaction time experiment using a 2^7 factorial design with 4 operators as blocks.

## Experimental Design

### Factors (k = 7):
1. **Minimum delay**: -1 = 0.5s, +1 = 2.0s
2. **Maximum delay**: -1 = 3.0s, +1 = 7.0s
3. **Frame motion**: -1 = static, +1 = shaking
4. **Background**: -1 = white, +1 = random colors
5. **Font scale**: -1 = small, +1 = large
6. **Text color**: -1 = black, +1 = random colors
7. **Text motion**: -1 = static, +1 = moving

### Blocks (Operators):
- Sabina
- Florentin
- Francisco
- Petr

### Response:
- Reaction time (ms) - 10 repetitions per condition

## 1. Setup and Data Loading

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.outliers_influence import OLSInfluence
import warnings
warnings.filterwarnings('ignore')

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("Libraries loaded successfully")

Libraries loaded successfully


In [2]:
import os
from io import StringIO

# Define file paths with local-first approach and GitHub fallback
data_files = {
    'Florentin': 'reaction_time_measurements_Florentin.csv',
    'Francisco': 'reaction_time_measurements_Francisco.csv',
    'Sabina': 'reaction_time_measurements_Sabina.csv',
    'Petr': 'reaction_time_petr.csv'
}

base_url = 'https://raw.githubusercontent.com/VojtasekP/NAEX_reaction_time/main/'

dfs = []
for operator, filename in data_files.items():
    local_path = filename
    if os.path.exists(local_path):
        # Read raw content first to handle malformed files
        with open(local_path, 'r') as f:
            content = f.read()
        print(f"Loaded {operator} from local file: {local_path}")
    else:
        try:
            import urllib.request
            url = base_url + filename
            with urllib.request.urlopen(url) as response:
                content = response.read().decode('utf-8')
            print(f"Loaded {operator} from GitHub: {url}")
        except Exception as e:
            print(f"Error loading {operator}: {e}")
            continue

    # Fix malformed Sabina file: first line has multiple records concatenated
    # Split by the header pattern and reconstruct
    lines = content.split('\n')
    header = 'timestamp_utc,participant,stimulus_type,font_size,motion,input_mode,frame_motion,shaking,text_color_mode,background_mode,min_delay_sec,max_delay_sec,repetitions,trial,stimulus,reaction_time_ms'

    # Check if first line is malformed (contains multiple records)
    if lines[0].count('timestamp_utc') > 1 or lines[0].count(',') > 20:
        print(f"  -> Fixing malformed first line in {operator}'s file...")
        # The first line might have header + multiple data records concatenated
        # Split by the date pattern to separate records
        import re
        first_line = lines[0]
        # Extract header
        fixed_lines = [header]
        # Find all records (they start with date pattern)
        records = re.split(r'(?=\d{4}-\d{2}-\d{2}T)', first_line)
        for rec in records:
            if rec.strip() and not rec.startswith('timestamp'):
                fixed_lines.append(rec.strip())
        # Add remaining lines
        for line in lines[1:]:
            if line.strip():
                fixed_lines.append(line.strip())
        content = '\n'.join(fixed_lines)

    # Parse CSV
    df = pd.read_csv(StringIO(content))

    # Ensure participant column is consistent
    df['participant'] = operator
    dfs.append(df)
    print(f"  -> {len(df)} rows loaded for {operator}")

# Combine all data
df_raw = pd.concat(dfs, ignore_index=True)
print(f"\nTotal rows: {len(df_raw)}")
print(f"Columns: {list(df_raw.columns)}")
print(f"\nRows per operator:")
print(df_raw['participant'].value_counts())

Error loading Florentin: HTTP Error 404: Not Found
Error loading Francisco: HTTP Error 404: Not Found
Error loading Sabina: HTTP Error 404: Not Found
Error loading Petr: HTTP Error 404: Not Found


ValueError: No objects to concatenate

In [ ]:
# Display first few rows
df_raw.head(10)

## 2. Data Preprocessing and Factor Coding

Convert the experimental settings to coded factor levels (-1, +1).

In [ ]:
# Create a copy for analysis
df = df_raw.copy()

# Code factors according to desc_factors.txt
# Factor 1: minimum delay (-1 = 0.5, +1 = 2.0)
df['A_min_delay'] = df['min_delay_sec'].apply(lambda x: -1 if x < 1.0 else 1)

# Factor 2: maximum delay (-1 = 3, +1 = 7)
df['B_max_delay'] = df['max_delay_sec'].apply(lambda x: -1 if x < 5.0 else 1)

# Factor 3: frame motion (-1 = static, +1 = shaking)
df['C_frame_motion'] = df['frame_motion'].apply(lambda x: 1 if 'Shaking' in str(x) else -1)

# Factor 4: background (-1 = white, +1 = random colors)
df['D_background'] = df['background_mode'].apply(lambda x: 1 if 'Random' in str(x) else -1)

# Factor 5: font scale (-1 = small, +1 = large)
df['E_font_scale'] = df['font_size'].apply(lambda x: 1 if 'Large' in str(x) else -1)

# Factor 6: text color (-1 = black, +1 = random)
df['F_text_color'] = df['text_color_mode'].apply(lambda x: 1 if 'Random' in str(x) else -1)

# Factor 7: text motion (-1 = static, +1 = moving)
df['G_text_motion'] = df['motion'].apply(lambda x: 1 if 'Moving' in str(x) else -1)

# Block: Operator/Participant
df['Block'] = df['participant']

# Response: reaction time
df['Y'] = df['reaction_time_ms']

print("Factor coding complete")
print("\nFactor summary:")
for col in ['A_min_delay', 'B_max_delay', 'C_frame_motion', 'D_background', 'E_font_scale', 'F_text_color', 'G_text_motion']:
    print(f"{col}: {df[col].value_counts().to_dict()}")

In [ ]:
# Create unique experimental condition identifier
df['condition'] = (df['A_min_delay'].astype(str) + '_' +
                   df['B_max_delay'].astype(str) + '_' +
                   df['C_frame_motion'].astype(str) + '_' +
                   df['D_background'].astype(str) + '_' +
                   df['E_font_scale'].astype(str) + '_' +
                   df['F_text_color'].astype(str) + '_' +
                   df['G_text_motion'].astype(str))

# Count unique conditions per block
print("Unique conditions per block:")
print(df.groupby('Block')['condition'].nunique())

print(f"\nTotal unique conditions: {df['condition'].nunique()}")

## 3. Aggregate Data by Condition

Average reaction times over the 10 repetitions per condition.

In [ ]:
# Aggregate: mean reaction time per condition per block
factors = ['A_min_delay', 'B_max_delay', 'C_frame_motion', 'D_background',
           'E_font_scale', 'F_text_color', 'G_text_motion', 'Block']

df_agg = df.groupby(factors).agg(
    Y_mean=('Y', 'mean'),
    Y_std=('Y', 'std'),
    Y_count=('Y', 'count')
).reset_index()

print(f"Aggregated data shape: {df_agg.shape}")
print(f"\nConditions per block:")
print(df_agg.groupby('Block').size())

df_agg.head(10)

## 4. Exploratory Data Analysis

In [ ]:
# Summary statistics by block
print("Summary statistics by Block (Operator):")
print(df_agg.groupby('Block')['Y_mean'].describe())

In [ ]:
# Box plot of reaction times by block
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# By block - explicit order
block_order = ['Florentin', 'Francisco', 'Petr', 'Sabina']
sns.boxplot(x='Block', y='Y_mean', data=df_agg, order=block_order, ax=axes[0])
axes[0].set_title('Reaction Time by Block (Operator)')
axes[0].set_xlabel('Operator')
axes[0].set_ylabel('Mean Reaction Time (ms)')

# Distribution
df_agg['Y_mean'].hist(bins=30, ax=axes[1], edgecolor='black')
axes[1].set_title('Distribution of Mean Reaction Times')
axes[1].set_xlabel('Mean Reaction Time (ms)')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

# Commentary on EDA
print("""
INTERPRETATION - Exploratory Data Analysis:
================================================================================
1. OPERATOR VARIABILITY:
   - Boxplots show differences in reaction times between operators
   - Some differences may be statistically significant (see ANOVA below)
   - Blocking by operator is justified - eliminates inter-personal variability

2. DISTRIBUTION:
   - Histogram shows slightly right-skewed distribution (typical for RT data)
   - Most reaction times are between 900-1400 ms
   - Some extremes (>2000 ms) may indicate attention loss or errors
""")

In [ ]:
# Main effects visualization
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

factor_names = ['A_min_delay', 'B_max_delay', 'C_frame_motion', 'D_background',
                'E_font_scale', 'F_text_color', 'G_text_motion']

factor_labels = ['Min Delay', 'Max Delay', 'Frame Motion', 'Background',
                 'Font Scale', 'Text Color', 'Text Motion']

for i, (factor, label) in enumerate(zip(factor_names, factor_labels)):
    ax = axes[i // 4, i % 4]
    means = df_agg.groupby(factor)['Y_mean'].mean()
    ax.bar(means.index.astype(str), means.values, color=['steelblue', 'darkorange'])
    ax.set_title(f'{label}')
    ax.set_xlabel('Level')
    ax.set_ylabel('Mean RT (ms)')
    ax.set_xticks([0, 1])
    ax.set_xticklabels(['-1', '+1'])

# Hide the last empty subplot
axes[1, 3].set_visible(False)

plt.suptitle('Main Effects Plot', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 5. Calculate Effects

Calculate main effects and interaction effects for the 2^7 factorial design.

In [ ]:
def calculate_effects(df, response_col, factors):
    """
    Calculate main effects and two-factor interactions for 2^k factorial design.
    """
    effects = {}

    # Grand mean
    grand_mean = df[response_col].mean()
    effects['Intercept'] = grand_mean

    # Main effects
    for factor in factors:
        high = df[df[factor] == 1][response_col].mean()
        low = df[df[factor] == -1][response_col].mean()
        effects[factor] = (high - low) / 2

    # Two-factor interactions
    for i, f1 in enumerate(factors):
        for f2 in factors[i+1:]:
            interaction_col = df[f1] * df[f2]
            high = df[interaction_col == 1][response_col].mean()
            low = df[interaction_col == -1][response_col].mean()
            effects[f'{f1}:{f2}'] = (high - low) / 2

    return effects

# Calculate effects
factor_cols = ['A_min_delay', 'B_max_delay', 'C_frame_motion', 'D_background',
               'E_font_scale', 'F_text_color', 'G_text_motion']

effects = calculate_effects(df_agg, 'Y_mean', factor_cols)

# Create DataFrame for display
effects_df = pd.DataFrame(list(effects.items()), columns=['Effect', 'Value'])
effects_df['Absolute'] = effects_df['Value'].abs()
effects_df = effects_df.sort_values('Absolute', ascending=False).reset_index(drop=True)

print("Effects (sorted by absolute value):")
print(effects_df.head(20))

In [ ]:
# Half-normal probability plot with reference line
effects_no_intercept = effects_df[effects_df['Effect'] != 'Intercept'].copy()
effects_sorted = effects_no_intercept.sort_values('Absolute')

n = len(effects_sorted)
i = np.arange(1, n + 1)
probabilities = (i - 0.5) / n
expected_values = stats.halfnorm.ppf(probabilities)

plt.figure(figsize=(12, 8))
plt.scatter(effects_sorted['Absolute'], expected_values, s=60, alpha=0.7)

# Add reference line (fitted through first 50% of effects)
cutoff = int(0.5 * len(effects_sorted))
from scipy.stats import linregress
slope, intercept, _, _, _ = linregress(effects_sorted['Absolute'].iloc[:cutoff].values, expected_values[:cutoff])
x_line = np.array([effects_sorted['Absolute'].min(), effects_sorted['Absolute'].max()])
y_line = slope * x_line + intercept
plt.plot(x_line, y_line, 'r--', alpha=0.5, linewidth=2, label='Reference line')

# Label points that deviate significantly from the line
significant_effects = []
for idx, row in effects_sorted.iterrows():
    x_val = row['Absolute']
    y_actual = expected_values[effects_sorted.index.get_loc(idx)]
    y_expected = slope * x_val + intercept
    # Label if deviation is significant (above line)
    if y_actual > y_expected + 0.3 or x_val > effects_sorted['Absolute'].quantile(0.8):
        plt.annotate(row['Effect'], (x_val, y_actual),
                     textcoords="offset points", xytext=(5, 5),
                     fontsize=9, ha='left')
        significant_effects.append(row['Effect'])

plt.xlabel('Absolute Effect', fontsize=12)
plt.ylabel('Half-Normal Probability', fontsize=12)
plt.title('Half-Normal Probability Plot of Effects', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Commentary
print("""
INTERPRETATION - Half-Normal Plot:
================================================================================
PRINCIPLE:
- Points ON the reference line = inactive factors (noise)
- Points significantly ABOVE the line = active (significant) factors

IDENTIFIED ACTIVE EFFECTS:
""")
for eff in significant_effects:
    val = effects_df[effects_df['Effect'] == eff]['Value'].values[0]
    print(f"  • {eff}: {val:+.2f} ms")
print("""
NOTE:
- Reference line fitted through bottom 50% of effects (assumed noise)
- Effects deviating significantly from line are candidates for significant factors
""")

## 6. ANOVA with Blocking

Perform ANOVA including block effects (operators).

In [ ]:
# Full model with block effect
# Using main effects + block only first
formula_main = 'Y_mean ~ C(Block) + A_min_delay + B_max_delay + C_frame_motion + D_background + E_font_scale + F_text_color + G_text_motion'

model_main = ols(formula_main, data=df_agg).fit()

print("=" * 60)
print("ANOVA TABLE - Main Effects + Block")
print("=" * 60)
anova_main = anova_lm(model_main, typ=2)
print(anova_main.round(4))

# Commentary
print("""
INTERPRETATION - ANOVA Results:
================================================================================
SIGNIFICANCE (p < 0.05):
""")
for idx in anova_main.index:
    if idx != 'Residual':
        p_val = anova_main.loc[idx, 'PR(>F)']
        f_val = anova_main.loc[idx, 'F']
        significance = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else ""
        status = "SIGNIFICANT" if p_val < 0.05 else "not significant"
        print(f"  {idx}: F={f_val:.2f}, p={p_val:.4f} {significance} -> {status}")

print("""
NOTES:
- C(Block) = operator effect (blocking factor)
- Type II ANOVA: tests each factor after accounting for others
- High F = large difference between factor levels relative to noise
""")

In [ ]:
# Model with significant two-factor interactions
# Based on effect analysis, include most important interactions
formula_full = '''Y_mean ~ C(Block) + A_min_delay + B_max_delay + C_frame_motion + D_background +
                  E_font_scale + F_text_color + G_text_motion +
                  A_min_delay:B_max_delay + A_min_delay:C_frame_motion + A_min_delay:D_background +
                  B_max_delay:C_frame_motion + C_frame_motion:D_background + E_font_scale:G_text_motion'''

model_full = ols(formula_full, data=df_agg).fit()

print("=" * 60)
print("ANOVA TABLE - With Key Interactions")
print("=" * 60)
anova_full = anova_lm(model_full, typ=2)
print(anova_full.round(4))

In [ ]:
# Model summary
print("=" * 60)
print("MODEL SUMMARY")
print("=" * 60)
print(model_full.summary())

## 7. Model Diagnostics

Check model assumptions using studentized residuals.

In [ ]:
# Get studentized residuals
influence = OLSInfluence(model_full)
studentized_resid = influence.resid_studentized_internal
fitted_values = model_full.fittedvalues

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Residuals vs Fitted
axes[0, 0].scatter(fitted_values, studentized_resid, alpha=0.6)
axes[0, 0].axhline(y=0, color='r', linestyle='--')
axes[0, 0].axhline(y=2, color='gray', linestyle=':', alpha=0.5)
axes[0, 0].axhline(y=-2, color='gray', linestyle=':', alpha=0.5)
axes[0, 0].set_xlabel('Fitted Values')
axes[0, 0].set_ylabel('Studentized Residuals')
axes[0, 0].set_title('Studentized Residuals vs Fitted Values')

# 2. Q-Q Plot
stats.probplot(studentized_resid, dist="norm", plot=axes[0, 1])
axes[0, 1].set_title('Normal Q-Q Plot of Studentized Residuals')

# 3. Scale-Location
axes[1, 0].scatter(fitted_values, np.sqrt(np.abs(studentized_resid)), alpha=0.6)
axes[1, 0].set_xlabel('Fitted Values')
axes[1, 0].set_ylabel('√|Studentized Residuals|')
axes[1, 0].set_title('Scale-Location Plot')

# 4. Histogram of residuals
axes[1, 1].hist(studentized_resid, bins=30, edgecolor='black', density=True)
x_range = np.linspace(studentized_resid.min(), studentized_resid.max(), 100)
axes[1, 1].plot(x_range, stats.norm.pdf(x_range), 'r-', lw=2, label='Normal')
axes[1, 1].set_xlabel('Studentized Residuals')
axes[1, 1].set_ylabel('Density')
axes[1, 1].set_title('Distribution of Studentized Residuals')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# Normality test
shapiro_stat, shapiro_p = stats.shapiro(studentized_resid[:50])  # Shapiro-Wilk for first 50

# Count outliers
n_outliers = np.sum(np.abs(studentized_resid) > 2)

print("""
INTERPRETATION - Model Diagnostics:
================================================================================
1. RESIDUALS VS FITTED:
   - Looking for random scatter around zero
   - Pattern (funnel, curve) = assumption violation
   - Points outside ±2 = potential outliers

2. Q-Q PLOT:
   - Points on diagonal = normality OK
   - Deviations at tails = heavy tails / skewness

3. SCALE-LOCATION:
   - Constant variability = horizontal band
   - Increasing trend = heteroscedasticity

4. HISTOGRAM:
   - Comparison with normal distribution
""")
print(f"STATISTICS:")
print(f"  - Number of outliers (|r| > 2): {n_outliers} out of {len(studentized_resid)}")
print(f"  - Shapiro-Wilk test (subsample): W={shapiro_stat:.4f}, p={shapiro_p:.4f}")
if shapiro_p > 0.05:
    print(f"    -> Normality NOT rejected (p > 0.05)")
else:
    print(f"    -> Normality REJECTED (p < 0.05) - but ANOVA is robust")

In [ ]:
# Residuals by block - ensure all operators are shown
df_agg['residuals'] = studentized_resid

# Define explicit order for blocks
block_order = ['Florentin', 'Francisco', 'Petr', 'Sabina']

fig, ax = plt.subplots(figsize=(10, 6))
sns.boxplot(x='Block', y='residuals', data=df_agg, order=block_order, ax=ax)
ax.axhline(y=0, color='r', linestyle='--')
ax.axhline(y=2, color='gray', linestyle=':', alpha=0.5, label='±2 threshold')
ax.axhline(y=-2, color='gray', linestyle=':', alpha=0.5)
ax.set_title('Studentized Residuals by Block (Operator)')
ax.set_xlabel('Operator')
ax.set_ylabel('Studentized Residuals')
ax.legend()
plt.tight_layout()
plt.show()

# Check residuals statistics per block
print("\nResidual Statistics per Block:")
print("=" * 60)
resid_stats = df_agg.groupby('Block')['residuals'].agg(['mean', 'std', 'min', 'max', 'count'])
print(resid_stats.round(3))

print("""
INTERPRETATION - Residuals by Block:
================================================================================
- Residuals should be centered around 0 for each block (balanced design)
- Similar spread across blocks = homoscedasticity within blocks
- Outliers (|r| > 2) indicate unusual observations
- If one block has systematically different residuals, blocking was effective
""")

## 8. Pareto Chart of Effects

In [ ]:
# Pareto chart of standardized effects
# Use t-values from the model
t_values = model_main.tvalues.drop('Intercept')
t_df = pd.DataFrame({'Effect': t_values.index, 't_value': t_values.values})
t_df['abs_t'] = t_df['t_value'].abs()
t_df = t_df.sort_values('abs_t', ascending=True)

# Critical t-value for significance (alpha = 0.05)
df_residual = model_main.df_resid
t_crit = stats.t.ppf(0.975, df_residual)

plt.figure(figsize=(10, 8))
colors = ['steelblue' if abs(t) < t_crit else 'darkorange' for t in t_df['t_value']]
plt.barh(t_df['Effect'], t_df['abs_t'], color=colors)
plt.axvline(x=t_crit, color='r', linestyle='--', label=f't_crit = {t_crit:.2f} (α=0.05)')
plt.xlabel('|t-value|')
plt.ylabel('Effect')
plt.title('Pareto Chart of Standardized Effects')
plt.legend()
plt.tight_layout()
plt.show()

print("""
INTERPRETATION - Pareto Chart:
================================================================================
- Bars extending BEYOND the red line are statistically significant
- Orange bars = significant effects (p < 0.05)
- Blue bars = non-significant effects

The Pareto chart ranks effects by importance (standardized effect size).
Effects at the top of the chart have the largest impact on reaction time.

This visualization follows the Pareto principle - often a few factors
(20%) account for most of the variation (80%).
""")

## 9. Block Effect Analysis

In [ ]:
# Compare operators
print("\nBlock (Operator) Effect Analysis:")
print("=" * 60)

block_means = df_agg.groupby('Block')['Y_mean'].agg(['mean', 'std', 'count'])
print("\nMean Reaction Time by Operator:")
print(block_means.round(2))

# Tukey HSD for block comparison
tukey_result = pairwise_tukeyhsd(df_agg['Y_mean'], df_agg['Block'], alpha=0.05)
print("\nTukey HSD Multiple Comparison:")
print(tukey_result)

print("""
INTERPRETATION - Block Effect:
================================================================================
The block effect captures systematic differences between operators.

KEY OBSERVATIONS:
- Operators may differ in baseline reaction speed
- These differences are removed from experimental error by blocking
- Tukey HSD identifies which pairs of operators differ significantly

IF block effect is significant:
  -> Blocking was effective and reduced experimental error
  -> Individual differences exist but don't confound factor effects
""")

In [ ]:
# Visualization of Tukey results - manual plot (more robust than plot_simultaneous)
fig, ax = plt.subplots(figsize=(12, 6))

# Get unique groups and their means
block_means = df_agg.groupby('Block')['Y_mean'].mean().sort_values()
block_stds = df_agg.groupby('Block')['Y_mean'].std()
block_counts = df_agg.groupby('Block')['Y_mean'].count()

# Calculate confidence intervals (95%)
from scipy.stats import t as t_dist
alpha = 0.05

# Plot means with confidence intervals
y_positions = np.arange(len(block_means))
colors = plt.cm.Set2(np.linspace(0, 1, len(block_means)))

for i, (block, mean_val) in enumerate(block_means.items()):
    std = block_stds[block]
    n = block_counts[block]
    se = std / np.sqrt(n)
    t_crit = t_dist.ppf(1 - alpha/2, n - 1)
    ci = t_crit * se

    ax.errorbar(mean_val, i, xerr=ci, fmt='o', markersize=10,
                capsize=5, capthick=2, color=colors[i], linewidth=2,
                label=f'{block}: {mean_val:.1f} ± {ci:.1f} ms')

ax.set_yticks(y_positions)
ax.set_yticklabels(block_means.index)
ax.set_xlabel('Mean Reaction Time (ms)', fontsize=12)
ax.set_ylabel('Operator', fontsize=12)
ax.set_title('Operator Comparison: Mean Reaction Time with 95% CI', fontsize=14)
ax.legend(loc='upper right')
ax.grid(True, axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# Show Tukey summary table
print("\nTukey HSD Summary Table:")
print("=" * 70)
tukey_summary = tukey_result.summary()
print(tukey_summary)

## 10. Interpretation and Conclusions

In [ ]:
# Summarize significant effects
print("=" * 60)
print("SUMMARY OF SIGNIFICANT EFFECTS")
print("=" * 60)

# Get p-values for main effects
anova_main_clean = anova_main.copy()
anova_main_clean['significant'] = anova_main_clean['PR(>F)'] < 0.05

print("\nSignificant factors (p < 0.05):")
significant = anova_main_clean[anova_main_clean['significant']]
for idx in significant.index:
    if idx != 'Residual':
        p_val = anova_main_clean.loc[idx, 'PR(>F)']
        f_val = anova_main_clean.loc[idx, 'F']
        print(f"  {idx}: F = {f_val:.2f}, p = {p_val:.4f}")

print("\nNon-significant factors (p >= 0.05):")
non_significant = anova_main_clean[~anova_main_clean['significant']]
for idx in non_significant.index:
    if idx != 'Residual':
        p_val = anova_main_clean.loc[idx, 'PR(>F)']
        f_val = anova_main_clean.loc[idx, 'F']
        print(f"  {idx}: F = {f_val:.2f}, p = {p_val:.4f}")

print("""
INTERPRETATION:
================================================================================
SIGNIFICANT EFFECTS indicate factors that reliably influence reaction time.
- These should be controlled/optimized in the experiment
- Their settings matter for achieving desired reaction time

NON-SIGNIFICANT EFFECTS can be set for convenience or cost savings.
- These factors don't strongly influence reaction time
- Can be held at any level without major impact
""")

In [ ]:
# Effect sizes for significant factors
print("=" * 60)
print("EFFECT SIZES (Cohen's f)")
print("=" * 60)

# Calculate partial eta-squared and Cohen's f
SS_total = anova_main['sum_sq'].sum()
SS_error = anova_main.loc['Residual', 'sum_sq']

print("\n{:<25} {:>12} {:>12} {:>15}".format("Effect", "η² partial", "Cohen's f", "Magnitude"))
print("-" * 65)

for idx in anova_main.index:
    if idx != 'Residual':
        SS_effect = anova_main.loc[idx, 'sum_sq']
        partial_eta_sq = SS_effect / (SS_effect + SS_error)
        cohens_f = np.sqrt(partial_eta_sq / (1 - partial_eta_sq))

        # Determine magnitude
        if cohens_f < 0.10:
            magnitude = "negligible"
        elif cohens_f < 0.25:
            magnitude = "small"
        elif cohens_f < 0.40:
            magnitude = "medium"
        else:
            magnitude = "LARGE"

        print(f"{idx:<25} {partial_eta_sq:>12.4f} {cohens_f:>12.4f} {magnitude:>15}")

print("""
INTERPRETATION - Effect Sizes:
================================================================================
Cohen's f benchmarks:
  - Small:   f = 0.10 (1% variance explained)
  - Medium:  f = 0.25 (6% variance explained)
  - Large:   f = 0.40 (14% variance explained)

PRACTICAL SIGNIFICANCE:
- Large effects are practically important regardless of p-value
- Small significant effects may not be worth optimizing
- Effect size helps prioritize which factors to control
""")

## 11. Interaction Plots

In [ ]:
# Create interaction plots for top interactions
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Define interactions to plot
interactions = [
    ('A_min_delay', 'B_max_delay'),
    ('A_min_delay', 'C_frame_motion'),
    ('C_frame_motion', 'D_background'),
    ('E_font_scale', 'G_text_motion'),
    ('B_max_delay', 'C_frame_motion'),
    ('A_min_delay', 'D_background')
]

factor_labels_dict = {
    'A_min_delay': 'Min Delay', 'B_max_delay': 'Max Delay',
    'C_frame_motion': 'Frame Motion', 'D_background': 'Background',
    'E_font_scale': 'Font Scale', 'F_text_color': 'Text Color',
    'G_text_motion': 'Text Motion'
}

for i, (f1, f2) in enumerate(interactions):
    ax = axes[i // 3, i % 3]

    # Calculate means for each combination
    means = df_agg.groupby([f1, f2])['Y_mean'].mean().unstack()

    for col in means.columns:
        label = f'{factor_labels_dict[f2]}={col}'
        ax.plot(means.index, means[col], marker='o', label=label, linewidth=2, markersize=8)

    ax.set_xlabel(factor_labels_dict[f1])
    ax.set_ylabel('Mean RT (ms)')
    ax.set_title(f'{factor_labels_dict[f1]} × {factor_labels_dict[f2]}')
    ax.legend()
    ax.set_xticks([-1, 1])
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("""
INTERPRETATION - Interaction Plots:
================================================================================
HOW TO READ:
- Parallel lines = NO interaction (additive effects)
- Non-parallel lines = INTERACTION present

EXAMPLES:
- If lines cross: Strong interaction - factor effect depends on other factor level
- If lines diverge: Synergistic or antagonistic interaction
- If lines are parallel: Factors act independently

PRACTICAL IMPLICATIONS:
- Interactions mean we cannot interpret main effects in isolation
- Optimal settings must consider factor combinations
""")

## 12. Final Conclusions

Based on the 2^7 factorial design analysis with blocking:

In [ ]:
# Final summary with interpretation
print("""
========================================================================
CONCLUSIONS - Reaction Time 2^7 Factorial Design with Blocking
========================================================================

EXPERIMENTAL DESIGN:
- 7 factors at 2 levels each (2^7 = 128 conditions)
- 4 blocks (operators): Sabina, Florentin, Francisco, Petr
- 32 unique conditions per operator
- 10 repetitions per condition
- Response: Reaction time (ms)

KEY FINDINGS FROM THIS ANALYSIS:

1. BLOCKING EFFECTIVENESS:
   - Block effect (C(Block)) captures operator-to-operator differences
   - If significant: blocking successfully removed nuisance variation
   - This improves power to detect true factor effects

2. MAIN EFFECTS:
   - Factors with p < 0.05 have significant impact on reaction time
   - Effect size (Cohen's f) indicates practical significance
   - Large effects (f > 0.40) are most important for optimization

3. INTERACTIONS:
   - Non-parallel lines in interaction plots indicate dependencies
   - Cannot optimize factors independently when interactions exist
   - Need to consider factor combinations for best settings

4. MODEL VALIDITY:
   - Studentized residuals should follow N(0,1) approximately
   - Q-Q plot deviations indicate non-normality
   - Outliers (|r| > 2) may need investigation

METHODOLOGY NOTES:
- Used aggregated data (mean over 10 repetitions) for analysis
- Type II ANOVA for balanced comparison
- Tukey HSD for multiple comparisons between operators
- Blocking removes operator-to-operator variation from error term

RECOMMENDATIONS:
- Focus on significant main effects for process improvement
- Consider significant interactions when setting factor levels
- Validate findings with confirmation runs

========================================================================
""")

# Print R-squared for model fit
print(f"Model Fit Statistics:")
print(f"  R-squared: {model_full.rsquared:.4f}")
print(f"  Adjusted R-squared: {model_full.rsquared_adj:.4f}")
print(f"  This means the model explains {model_full.rsquared*100:.1f}% of variance in reaction time.")